In [69]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [70]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/drive/My Drive/Kaggle'

In [ ]:
%cd /content/drive/My Drive/Kaggle

In [ ]:
!kaggle competitions download -c tabular-playground-series-jun-2021

In [ ]:
!unzip \*.zip && rm *.zip

In [150]:
train = pd.read_csv("/content/drive/My Drive/Kaggle/train.csv")
train.head()

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74,target
0,0,0,0,6,1,0,0,0,0,7,0,0,0,3,0,1,0,0,3,3,1,0,2,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,11,0,0,0,0,0,9,0,0,0,0,0,0,3,0,1,3,0,0,0,0,0,0,0,1,1,0,0,3,0,0,0,0,0,0,2,0,0,Class_6
1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,1,0,Class_6
2,2,0,0,0,0,0,1,0,3,0,0,1,0,0,0,0,0,0,0,0,5,4,0,0,0,0,0,0,0,0,1,0,0,0,0,2,0,0,5,0,5,0,1,0,0,0,1,0,0,1,0,7,0,1,1,0,3,4,0,0,1,3,0,2,0,0,8,0,0,0,0,1,0,0,0,0,Class_2
3,3,0,0,7,0,1,5,2,2,0,1,2,0,5,0,0,4,0,0,22,2,1,0,0,0,0,3,0,37,0,1,3,13,0,10,0,3,1,1,0,7,0,0,2,0,1,0,0,0,0,0,0,10,0,0,25,1,0,1,2,0,2,0,7,0,0,0,0,4,0,2,2,0,4,3,0,Class_8
4,4,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Class_2


In [160]:
test = pd.read_csv("/content/drive/MyDrive/Kaggle/test.csv")
test.head()

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,feature_50,feature_51,feature_52,feature_53,feature_54,feature_55,feature_56,feature_57,feature_58,feature_59,feature_60,feature_61,feature_62,feature_63,feature_64,feature_65,feature_66,feature_67,feature_68,feature_69,feature_70,feature_71,feature_72,feature_73,feature_74
0,200000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,200001,1,2,0,0,0,0,0,0,0,0,1,0,2,2,0,0,1,0,0,0,2,0,0,1,0,3,0,1,0,0,2,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,1,3,2,0,0,0,0,0,0,1,7,0,1,3,1,3,0,0,0,0,3,0,0
2,200002,0,1,7,1,0,0,0,0,6,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,2,0,0,4,0,0,0,0,0,0,0,0,1,0,0,4,0,0,1,0,0,0,6,0,0,2,10,0,0,0,2,1,0,0,0,0,0,3,0,0,0,0,3,0,2,0,0
3,200003,0,0,0,4,3,1,0,0,0,0,7,0,0,0,7,0,0,0,5,0,1,0,0,0,0,0,0,2,0,0,1,0,0,0,0,0,0,0,0,1,0,0,7,5,0,0,0,0,0,0,0,0,0,2,3,0,0,0,1,0,0,0,0,0,4,0,0,0,1,0,0,0,4,0,0
4,200004,0,0,5,0,0,0,0,0,0,8,1,0,2,0,0,0,0,0,9,1,0,0,0,0,0,0,0,0,3,0,5,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,2,2,1,5,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0


In [152]:
sample_submission = pd.read_csv("/content/drive/MyDrive/Kaggle/sample_submission.csv")
sample_submission.head()

,id,Class_1,Class_2,Class_3,Class_4,Class_5,Class_6,Class_7,Class_8,Class_9
0,200000,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111
1,200001,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111
2,200002,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111
3,200003,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111
4,200004,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111


In [ ]:
test.describe()

In [ ]:
train.describe()

In [81]:
print('train : ',len(train))
print('test:  ',len(test))
print('sample_submission: ', len(sample_submission))

train :  200000
test:   100000
sample_submission:  100000


In [162]:

train.drop(['id'], axis =1, inplace = True)
x = train.drop(['target'], axis=1).values


In [163]:
# convert target values to 0,1,2,...
tar_dic = {
           'Class_1': 0,
           'Class_2': 1,
           'Class_3': 2,
           'Class_4': 3,
           'Class_5': 4,
           'Class_6': 5,
           'Class_7': 6,
           'Class_8': 7,
           'Class_9': 8
}

train['target'] = train['target'].map(tar_dic)
y = train['target'].values

In [84]:
#Splitting training data into train and validation set
x_train, x_val, y_train, y_val = train_test_split(x,y,test_size = 0.2)

In [110]:
print(x_train)
y_train.shape
type(y_train)

tensor([[-0.2473, -0.2927, -0.3428,  ..., -0.2997, -0.2724, -0.1614],
        [-0.2473, -0.2927, -0.1881,  ...,  0.1670,  3.9886,  0.8529],
        [-0.2473, -0.2927, -0.3428,  ..., -0.2997, -0.2724,  0.0922],
        ...,
        [-0.2473, -0.2927, -0.1881,  ..., -0.0664, -0.2724, -0.1614],
        [-0.2473, -0.2927, -0.3428,  ..., -0.2997, -0.2724, -0.1614],
        [-0.2473,  2.7269,  1.6683,  ...,  0.4003,  1.6728,  0.3457]])


torch.Tensor

In [111]:
print(y_train)

tensor([[7],
        [5],
        [0],
        ...,
        [5],
        [8],
        [5]])


In [112]:
#Importing relevant modules
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import collections

In [122]:
#Scaling data
sc = StandardScaler()

x_train = sc.fit_transform(x_train)
x_val = sc.fit_transform(x_val)

x_train = torch.from_numpy(x_train.astype(np.float32))
x_val = torch.from_numpy(x_val.astype(np.float32))
y_train = y_train.float()
y_val = y_val.float()

# Converting y_train to a column vector

y_train = y_train.view(y_train.shape[0], 1)
y_val = y_val.view(y_val.shape[0], 1)

In [132]:
class LogisticRegression(nn.Module):
  def __init__(self, input_features):
    super().__init__()
    self.linear = nn.Linear(input_features, 1)

  def forward(self, x):
    x = self.linear(x)
    y_predicted = torch.sigmoid(x)

    return y_predicted


In [133]:
model = LogisticRegression(75)

In [140]:
from torch import optim
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr =0.0075)

In [141]:
#y_train = y_train.type(torch.LongTensor)
type(y_predicted)
y_predicted.dtype

torch.float32

In [144]:
#training

epochs = 100
for e in range(epochs):

  y_predicted = model(x_train)
  loss = criterion(y_predicted, y_train)
  loss.backward()
  optimizer.step()
  optimizer.zero_grad()

  if (e+1)%10 ==0:
    print(f'epoch : {e+1}, loss = {loss.item():.4f}')

with torch.no_grad():
  y_predicted = model(x_val)
  y_predicted_cls = y_predicted.round()
  acc = y_predicted_cls.eq(y_val).sum()/float(y_val.shape[0])
  print(f'accuracy = {acc:.4f}')

epoch : 10, loss = -397.7096
epoch : 20, loss = -397.7096
epoch : 30, loss = -397.7096
epoch : 40, loss = -397.7096
epoch : 50, loss = -397.7096
epoch : 60, loss = -397.7101
epoch : 70, loss = -397.7096
epoch : 80, loss = -397.7096
epoch : 90, loss = -397.7075
epoch : 100, loss = -397.7096
accuracy = 0.1220
